<a href="https://colab.research.google.com/github/SharmaKanishkaa/Analyzing-Spooky-Authors-with-NLP/blob/main/Spooky_NLP_and_Topic_Modelling_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

![](https://png.pngtree.com/thumb_back/fw800/background/20230615/pngtree-halloween-cuties-wallpaper-2017-image_2947813.jpg)

In this notebook, I shall conduct a very basic attempt at topic modelling this Spooky Author dataset. Topic modelling is the process in which we try uncover abstract themes or "topics" based on the underlying documents and words in a corpus of text. I will introduce two standard topic modelling techniques here with the first technique known as Latent Dirichlet Allocation (LDA) and the second Non-negative Matrix Factorization (NMF). I will also take the opportunity to introduce some Natural Language Processing basics such as Tokenization, Stemming and vectorization of the raw text which should also hopefully come in handy when making predictions with learning models.

The outline of this notebook is as follows:

1. **Exploratory Data Analysis (EDA) and Wordclouds** - Analyzing the data by generating simple statistics such word frequencies over the different authors as well as plotting some wordclouds (with image masks).


2. **Natural Language Processing (NLP) with NLTK (Natural Language Toolkit) ** - Introducing basic text processing methods such as tokenizations, stop word removal, stemming and vectorizing text via term frequencies (TF) as well as the inverse document frequencies (TF-IDF)

3. **Topic Modelling with LDA and NNMF** - Implementing the two topic modelling techniques of Latent Dirichlet Allocation (LDA) and Non-negative Matrix Factorization (NMF).

So, are you brave enough to take the next step and continue reading through this ghastly notebook?

---

**"*Because they pop! Pop, pop! Pop, pop! Pop, pop, pop!*" **- Pennywise, The Dancing Clown

In [ ]:
import base64
import numpy as np
import pandas as pd

# Plotly imports
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# Other imports
from collections import Counter
import imageio
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Loading the training data with Pandas
train = pd.read_csv("../input/spooky-author-identification/train.zip", compression='zip', sep=',', quotechar='"')

---

# 1. The Authors and their works EDA

First step, let us take a look at a quick peek of what the first three rows in the data has in store for us and who exactly are the authors

In [ ]:
train.head()

According to the competition page there are three distinct author initials we have already been provided with a mapping of these initials to the actual author which is as follows:

*(Links to their Wikipedia page profiles if you click on their names)*

1. **[EAP - Edgar Allen Poe](https://en.wikipedia.org/wiki/Edgar_Allan_Poe)** : American writer who wrote poetry and short stories that revolved around tales of mystery and the grisly and the grim. Arguably his most famous work is the poem - "The Raven" and he is also widely considered the pioneer of the genre of the detective fiction.

2. **[HPL - HP Lovecraft](https://en.wikipedia.org/wiki/H._P._Lovecraft)** : Best known for authoring works of horror fiction, the stories that he is most celebrated for revolve around the fictional mythology of the infamous creature "Cthulhu" - a hybrid chimera mix of Octopus head and humanoid body with wings on the back.

3. **[MWS - Mary Shelley](https://en.wikipedia.org/wiki/Mary_Shelley)** : Seemed to have been involved in a whole panoply of literary pursuits - novelist, dramatist, travel-writer, biographer. She is most celebrated for the classic tale of Frankenstein where the scientist Frankenstein a.k.a "The Modern Prometheus" creates the Monster that comes to be associated with his name.

Next, let us take a look at how large the training data is:

In [ ]:
train.shape

## Summary statistics of the training set

Here we can visualize some basic statistics in the data, like the distribution of entries for each author. For this purpose, I will invoke the handy Plot.ly visualisation library and plot some simple bar plots. Unhide the cell below if you want to see the Plot.ly code.

In [ ]:
z = {'EAP': 'Edgar Allen Poe', 'MWS': 'Mary Shelley', 'HPL': 'HP Lovecraft'}
data = [go.Bar(x = train.author.map(z).unique(),
               y = train.author.value_counts().values,
               marker = dict(colorscale='Jet',
                            color = train.author.value_counts().values),
               text = 'Text entries attributed to Author')]

layout = go.Layout(title='Target variable distribution')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-car')

In [ ]:
all_words = train['text'].str.split(expand=True).unstack().value_counts()
data = [go.Bar(x=all_words.index.values[0:50],
               y=all_words.values[0:50],
               marker=dict(colorscale='Jet',
                           color=all_words.values[0:100]
                          ),
               text='word counts'
)]

layout = go.Layout(title = 'Top 50 (Uncleaned) word frecuencies in the training set')

fig = go.Figure(data = data, layout = layout)

py.iplot(fig, filename='basic-bar')

Notice anything odd about the words that appear in this word frequency plot? Do these words actually tell us much about the themes and concepts that Mary Shelley wants to portray to the reader in her stories?

These words are all so commonly occuring words which you could find just anywhere else. Not just in spooky stories and novels by our three authors but also in newspapers, kid book, religious texts - really almost every other english text. Therefore we must find some way to preprocess our dataset first to strip out all these commonly occurring words which do not bring much to the table.

## WordClouds to visualise each author's work

One very handy visualization tool for a data scientist when it comes to any sort of natural language processing is plotting "Word Cloud". A word cloud (as the name suggests) is an image that is made up of a mixture of distinct words which may make up a text or book and where the size of each word is proportional to its word frequency in that text (number of times the word appears). Here instead of dealing with an actual book or text, our words can simply be taken from the column "text"

**Store the text of each author in  a Python list**

We first create three different python lists that store the texts of Edgar Allen Poe, HP Lovecraft and Mary Shelley respectively as follows:

In [ ]:
eap = train[train.author=='EAP']['text'].values
hpl = train[train.author=='HPL']['text'].values
mws = train[train.author=='MWS']['text'].values

Next to create our wordclouds, I will import the python module "wordcloud".

In [ ]:
from wordcloud import WordCloud, STOPWORDS

But generating a normal wordcloud is rather boring so I would like to introduce to you a technique of importing pictures (something relevant) and using the outline of that picture as a mask for our wordclouds. Therefore the pictures that I have chosen are the ones I feel most representative for their authors:

1. ) The Raven for Edgar Allen Poe 2.) Octopus Cthulu-thingy for HP Lovecraft and 3.) Frankenstein for Mary Shelly

The way I am loading in the pictures on Kaggle is a sort of a feature hack although readers familiar to my work know this trick. I first derive the Base64 encoding of whatever images I want to use and then use that particular encoding and re-convert the picture back on the notebook. The cell below contains the Base64 encoding of the three images I will use but I have hidden them so that I do not pollute this notebook with just long streteches of text - unhide them if you want to see the encoding.

To decode the image, I import the "codecs" module and save it as a new image here in this notebook. Once the image is saved, I can simply load it as a mask as follows:

Finally plotting the word clouds via the following few lines (unhide to see the code):

In [ ]:
# Edgar Allen Poe wordcloud
plt.figure(figsize=(20,18))
wc = WordCloud(background_color='black', max_words=10000, stopwords=STOPWORDS, max_font_size=40)
wc.generate(" ".join(eap))
plt.title('Edgar Allen Poe', fontsize=20)
plt.imshow(wc)
plt.axis('off')

In [ ]:
# HP Lovecraft wordcloud
plt.figure(figsize=(20,18))
wc = WordCloud(background_color='black', max_words=10000, stopwords=STOPWORDS, max_font_size=40)
wc.generate(" ".join(hpl))
plt.title('HP Lovecraft')
plt.imshow(wc)
plt.axis('off')

In [ ]:
# Mary Shelley wordcloud
plt.figure(figsize=(20,18))
wc = WordCloud(background_color='black', stopwords=STOPWORDS, max_words=10000, max_font_size=40)
wc.generate(" ".join(mws))
plt.title('Mary Shelley')
plt.imshow(wc)
plt.axis('off')

Well there you have it, three separate word clouds, one for each of our spooky authors. Right off the bat, you can see from these word clouds some of the choice words that were favoured by the different authors.

For example, you can see the HP lovecraft favours words like "dream", "time", "strange", "past", "ancient" which seem to resonate with themes that the author was famous for, themes around the hidden psyche and esoteric nature of fate and chance as well as the infamous creature Cthulhu and mentions of ancient cults and rituals associated with it.

On the other hand, one can see that Mary Shelley's words revolve around primal instincts and themes of morality which range from the positive to negative ends of the spectrum, such as "friend", "fear", "hope", "spirit" etc. - themes which resonate in her works such as Frankenstein

However, as you can see from the word clouds, there are still a handful of words that seem to be quite out of place. Words such as "us", "go", "he" which seem to appear commonly every where in text

---

# 2. Natural Language Processing

![](https://s-media-cache-ak0.pinimg.com/originals/d8/ca/04/d8ca049fd1ad2d92818763e55c57f49a.jpg)

In almost all Natural Language Processing (the field that explores interactions between a computer and human languages) tasks that you will come across (be it topic modelling, or word clustering or document-text classification etc), one will generally always have to undergo these few pre-processing steps to convert the input raw text into a form that is readable by your model and the machine. You certainly can't expect to feed a Random Forest model a paragraph of words and expect it to immediately predict which author that paragraph came from. Behind the scenes, text pre-processing can be boiled down to these few simple steps:

1. **Tokenization** - Segregation of the text into its individual constitutent words.
2. **Stopwords** - Throw away any words that occur too frequently as its frequency of occurrence will not be useful in helping detecting relevant texts. (as an aside also consider throwing away words that occur very infrequently).
3. **Stemming**  - combine variants of words into a single parent word that still conveys the same meaning
4. **Vectorization** - Converting text into vector format. One of the simplest is the famous bag-of-words approach, where you create a matrix (for each document or text in the corpus). In the simplest form, this matrix stores word frequencies (word counts) and is often referred to as vectorization of the raw text.

**Natural Language Toolkit (NLTK)**: To make our Natural Language Processing endeavours more convenient, let me introduce to you one of the most handy toolkits that on NLP - the Natural Language Toolkit, also more commonly referred to as the  [NLTK](http://www.nltk.org/) module. To import the toolkit, it is as easier as:



In [ ]:
import nltk

## 2a. Tokenization

The concept of tokenization is the act of taking a sequence of characters (think of Python strings) in a given document and dicing it up into its individual constituent pieces, which are the eponymous "tokens" of this method. One could loosely think of them as singular words in a sentence. One could naively implement the "split( )" method on a string which separates it into a python list based on the identifier in the argument. It is actually not that trivial to

Here we split the first sentence of the text in the training data just on a space as follows:

In [ ]:
# Storing the first text element as a string
first_text = train.text.values[0]
print(first_text)
print('='*90)
print(first_text.split(' '))

However as you can see from this first attempt at tokenization, the segregation of the sentence into its individual elements (or terms) is not entirely accurate. As an example, look at the second element of the list which contains the term "process,". The punctuation mark (comma) has also been included and is being treated along with the word "process" as a term in itself. Ideally we would like the comma and the word to be in two different and separate elements of the list. Trying to do this with pure python list operations will be quite complex so this is where the NLTK library comes into play. There is a convenient method "word_tokenize( )" (TreebankWord tokenizer) which strips out singular words as well as punctuations into separate elements automatically as follows:

In [ ]:
first_text_list = nltk.word_tokenize(first_text)
print(first_text_list)

## 2b. Stopword Removal

As alluded to above stopwords are generally words that appear so commonly and at such a high frequency in the corpus that they don't actually contribute much to the learning or predictive process as a learning model would fail to distinguish it from other texts. Stopwords include terms such as "to" or "the" and therefore, it would be to our benefit to remove them during the pre-processing phase. Conveniently, NLTK comes with a predefined list of 153 english stopwords. (unhide the second cell below to see this list)

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
len(stopwords)

In [ ]:
print(stopwords)

To filter out stop words from our tokenized list of words, we can simply use a list comprehension as follows:

In [ ]:
first_text_list_cleaned = [word for word in first_text_list if word.lower() not in stopwords]
print(first_text_list_cleaned)
print('='*90)
print('Length of the original list: {0} words\n'
     'Length of the list after stopwords removal: {1} words'
     .format(len(first_text_list), len(first_text_list_cleaned)))

As you can see, our list with stopwords removed is now substantially shorter than before where commonly occuring words such as "I", "me", "to" and "the" have been removed.

## 2c. Stemming and Lemmatization

After removal of stopwords, the next stage of NLP that I would like to introduce is the process of Stemming.  The work at this stage attempts to reduce as many different variations of similar words into a single term ( different branches all reduced to single word *stem*). Therefore if we have "running", "runs" and "run", you would really want these three distinct words to collapse into just the word "run". (However of course you lose granularity of the past, present or future tense).

We can turn to NLTK again which provides various stemmers which include variants such as the Porter stemming algorithm, the lancaster stemmer and the Snowball stemmer. In the following example, I will create a porter stemmer instance as follows:

In [ ]:
stemmer = nltk.stem.PorterStemmer()

And now we can use stemmer to see if it can reduce our these test words ("running", "runs", "run") into their a single stemmed word. Conveniently we can test the stemmer on the fly as follows:

In [ ]:
print('The stemmed form of running is: {}'.format(stemmer.stem('running')))
print('The stemmed form of runs is: {}'.format(stemmer.stem('runs')))
print('The stemmed form of run is: {}'.format(stemmer.stem('run')))

As we can see, the stemmer has successfully reduced the given words above into a base form and this will be most in helping us reduce the size of our dataset of words when we come to learning and classification tasks.

However there is one flaw with stemming and that is the fact that the process involves quite a [crude heuristic in chopping off the ends of words](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html) in the hope of reducing a particular word into a human recognizable base form. Therefore this process does not take into account vocabulary or word forms when collapsing words as this example will illustrate:

In [ ]:
print('The stemmed form of leaves is: {}'.format(stemmer.stem('leaves')))

### Lemmatization to the rescue

Therefore we turn to another that we could use in lieu of stemming. This method is called lemmatization which aims to achieve the same effect as the former method. However unlike a stemmer, lemmatizing the dataset aims to reduce words based on an actual dictionary or vocabulary (the Lemma) and therefore will not chop off words into stemmed forms that do not carry any lexical meaning. Here we can utilize NLTK once again to initialize a lemmatizer (WordNet variant) and inspect how it collapses words as follows:

***Needed unzip in the current enviroment***

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()
print('The lemmatized form of leaves is: {}'.format(lemm.lemmatize('leaves')))

Brilliant, now we see that our lemmatizer is working and collapsing words into a form that makes much more lexical sense.

## 2d. Vectorizing Raw Text

In the vast collection of NLP literature, there are many different purposes for analyzing raw text, where in some cases you would like to compare the similarity of one body of text to another (Clustering techniques/Distance measurements), text classification (the purpose of this competition) as well as uncovering the topics that comprise a body of text (the aim of this notebook). With the purpose of uncovering topics at the back of our minds we must now think of how to feed the raw text into a machine learning model. Having already discussed tokenization, stopword removals and stemming (or maybe lemmatizing) we have now arrived at a reasonably cleaner text dataset then we started out with. However at this juncture, our raw text though human readable is still unfortunately not yet machine readable. A machine can read in bits and numbers and therefore we will first need to convert our text into numbers for which we utilise a very common approach known as the Bag-of-Words

**The Bag of Words approach**

This approach uses the counts of words as a starting block and records the occurrence of each word (from the entire text) in a vector specific to that particular word. For example given these two sentences "I love to eat Burgers", "I love to eat Fries", we first tokenize to obtain our vocabulary of 6 words from which we can get the word counts for - [I, love, to, eat, Burgers, Fries].

Vectorizing the text via the Bag of Words approach, we get six distinct vectors one for each word. So you ask since we now have rows consisting of numbers (instead of text) what forms the columns (or features)? Well each word now becomes an individual feature/column in this new transformed dataset. To illustrate this point, I shall utilize the Scikit-learn library to implement a vectorizer that generates a vector of word counts (term frequencies) - via the CountVectorizer method as follows.

In [ ]:
# Defining our sentence
sentence = ['I love to eat Burgers',
            'I love to eat Fries']
vectorizer = CountVectorizer(min_df=0)
sentence_transform = vectorizer.fit_transform(sentence)

**Fitting the vectorizer to the dataset**

Here we initialize and create a simple term frequency object via the CountVectorizer function simply called "vectorizer". The parameters that I have provided explicitly (the rest are left as default) are the bare minimum. Here "min_df" in the parameter refers to the minimum document frequency and the vectorizer will simply drop all words that occur less than that value set (either integer or in fraction form).  For a detailed read up on this method as well as the rest of the parameters that one could use, I refer you to the [Sklearn website](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

Finally we apply the fit_transform method is actually comprised of two steps. The first step is the fit method where the vectorizer is mapped to the the dataset that you provide. Once this is done,  the actual vectorizing operation is performed via the transform method where the raw text is turned into its vector form as shown below:

In [ ]:
print('The features are:\n {}'.format(vectorizer.get_feature_names_out()))
print('\nThe vectorized array looks like:\n {}'.format(sentence_transform.toarray()))

In [ ]:
sentence_transform

**Sparse matrix vector ouptuts**

From the output of the vectorized text, we can see that the features consist of the words in the corpus of text that we fed into the vectorizer (here the corpus being the two sentences we defined earlier). Simply call the get_feature_names_out attribute from the vectorizer to inspect it.

With regards to the transformed text, one would be tempted to inspect the values by simplying calling it. However when you try to call it you really just get a message which states "*sparse matrix of type class 'numpy.int64' with 8 stored elements in Compressed Sparse Row format*". Therefore this means that the vectorizer returns the transformed raw text as a matrix where most of its values are zero or almost negligible, hence the term sparse. Thinking about this, it does make sense that our returned matrices contain quite a high degree of sparsity.

---

# 3. Topic modelling

![](https://www.hillsboroughcounty.org/library/hillsborough/pumpkin-outside-nr.jpg?h=475&w=845&la=en&hash=B7A332CDADCCCA7909334E874F03A1E5)

Arriving at our *Final Destination* (pun intended), I will implement two different topic modelling techniques as follows:

1. **Latent Dirichlet Allocation** - Probabilistic, generative model which uncovers the topics latent to a dataset by assigning weights to words in a corpus, where each topic will assign different probability weights to each word.

2. **Non-negative Matrix Factorization** - Approximation method that takes an input matrix and approximates the factorization of this matrix into two other matrices, with the caveat that the values in the matrix be non-negative.



In [ ]:
# Define a helper function to print top words
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = '\nTopic #{}:'.format(index)
        message += ' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words -1 :-1]])
        print(message)
        print('='*70)

## 3a. Putting all the preprocessing steps together

This is now the perfect opportunity to piece together all the text preprocessing steps that was mentioned in the previous section. So you're asking yourself, do we really need to go through all the effort and steps again in defining tokenization, stopword removals, stemming/lemmatizing etc?

Thankfully we do not need to go through all of that again. I conveniently omitted a key detail about Sklearn vectorizers but will mention it at this juncture. When you vectorize the raw text with CountVectorizer, the dual stages of tokenizing and stopwords filtering are automatically included as a high-level component. Here unlike the NLTK  tokenizer that you were introduced to in the Section 2a earlier, Sklearn's tokenizer discards all single character terms like ('a', 'w' etc) and also lower cases all terms by default. Filtering out stopwords in Sklearn is as convenient as passing the value 'english' into the argument "stop_words" where a built-in English stopword list is automatically used.

Unfortunately, there is no built-in lemmatizer in the vectorizer so we are left with a couple of options. Either implementing it separately everytime before feeding the data for vectorizing or somehow extend the sklearn implementation to include this functionality. Luckily for us, we have the latter option where we can extend the CountVectorizer class by overwriting the "build_analyzer" method as follows:

### Extending the CountVectorizer class with a lemmatizer

In [ ]:
lemm = WordNetLemmatizer()
class LemmaCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(LemmaCountVectorizer, self).build_analyzer()
        return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))

Here we have utilised some subtle concepts from Object-Oriented Programming (OOP). We have essentially inherited and subclassed the original Sklearn's CountVectorizer class and overwritten the build_analyzer method by implementing the lemmatizer for each list in the raw text matrix.

In [ ]:
# Storing the entire training text in a list
text = list(train.text.values)
# Calling our overwritten Count vectorizer
tf_vectorizer = LemmaCountVectorizer(max_df=0.95,
                                    min_df=2,
                                    stop_words='english',
                                    decode_error='ignore')
tf = tf_vectorizer.fit_transform(text)

**Revisiting our Term frequencies**

Having implemented our lemmatized count vectorizer, let us revist the plots for the term frquencies of the top 50 words (by frequency). As you can see from the plot, all our prior preprocessing efforts have not gone to waste. With the removal of stopwords, the remaining words seem much more meaningful where you can see that all the stopwords in the earlier term frequency plot

In [ ]:
feature_names = tf_vectorizer.get_feature_names_out()
count_vec = np.asarray(tf.sum(axis=0)).ravel()
zipped = list(zip(feature_names, count_vec))
x, y = (list(x) for x in zip(*sorted(zipped, key=lambda x: x[1], reverse=True)))


# Plotting the Plot.ly plot for the Top 50 word frequencies
data = [go.Bar(x = x[0:50],
              y = y[0:50],
              marker = dict(colorscale='Jet',
                           color = y[0:50]),
              text='Word counts')]
layout = go.Layout(title='Top 50 word frequencies after preprocessing')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')


# Plotting the Plot.ly plot for the bottom 100 word frequences
data = [go.Bar(x = x[-100:],
              y = y[-100:],
              marker = dict(colorscale='Portland',
                           color=y[-100:]),
              text='Word counts')]
layout = go.Layout(title='Bottom 100 word frequences after preprocessing')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

## 3b. Latent Dirichlet Allocation

Finally we arrive on the subject of topic modelling and the implementation of a couple of unsupervised learning algorithms. The first method that I will touch upon is [Latent Dirichlet Allocation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Now there are a couple of different implements of this LDA algorithm but in this notebook, I will be using Sklearn's implementation. Another very well-known LDA implementation is Radim Rehurek's [gensim](https://radimrehurek.com/gensim/), so check it out as well.

**Corpus - Document - Word : Topic Generation**

In LDA, the modelling process revolves around three things: the text corpus, its collection of documents, D and the words W in the documents. Therefore the algorithm attempts to uncover K topics from this corpus via the following way (illustrated by the diagram)

![Three_Level Bayesian Model](http://scikit-learn.org/stable/_images/lda_model_graph.png)

* Model each topic, $\kappa$ via a Dirichlet prior distribution given by $\beta_{k}$


* Model each document d by another Dirichlet distribution parameterized by $\alpha$


* Subsequently for document d, we generate a topic via a multinomial distribution which we then backtrack and use to generate the correspondings words related to that topic via another multinomial distribution


The LDA algorithm first models documents via a mixture model of topics. From these topics, words are then assigned weights based on the probability distribution of these topics. It is this probabilistic assignment over words that allow a user of LDA to say how likely a particular word falls into a topic. Subsequently from the collection of words assigned to a particular topic, are we thus able to gain an insight as to what that topic may actually represent from a lexical point of view.

From a standard LDA model, there are really a few key parameters that we have to keep in mind and consider programmatically tuning before we invoke the model:
1. n_components: The number of topics that you specify to the model
2. $\alpha$ parameter: This is the dirichlet parameter that can be linked to the document topic prior
3. $\beta$ parameter: This is the dirichlet parameter linked to the topic word prior

To invoke the  algorithm, we simply create an LDA instance through the Sklearn's *LatentDirichletAllocation* function. The various parameters would ideally have been obtained through some sort of validation scheme. In this instance, the optimal value of n_components (or topic number) was found by conducting a KMeans + Latent Semantic Analysis Scheme whereby the number of Kmeans clusters and number of LSA dimensions were iterated through and the best silhouette mean score.

In [ ]:
lda = LatentDirichletAllocation(n_components=11, max_iter=5, learning_method='online', learning_offset=50, random_state=17)

In [ ]:
lda.fit(tf)

### Topics generated by LDA

We will utilise our helper function we defined earlier "print_top_words" to return the top 10 words attributed to each of the LDA generated topics. To select the number of topics, this is handled through the parameter n_components in the function.

In [ ]:
n_top_words = 10
print('\nTopics in LDA model: ')
tf_feature_names = tf_vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
first_topic = lda.components_[0]
second_topic = lda.components_[1]
third_topic = lda.components_[2]
fourth_topic = lda.components_[3]

In [ ]:
first_topic.shape

### Word Cloud visualizations of the topics

In [ ]:
first_topic_words = [tf_feature_names[i] for i in first_topic.argsort()[:-50 -1 : -1]]
second_topic_words = [tf_feature_names[i] for i in second_topic.argsort()[:-50 -1: -1]]
third_topic_words = [tf_feature_names[i] for i in third_topic.argsort()[:-50 -1: -1]]
fourth_topic_words = [tf_feature_names[i] for i in fourth_topic.argsort()[:-50 -1: -1]]

### Word cloud of First Topic

In [ ]:
# Generating the wordcloud with the values under the category dataframe
firstcloud = WordCloud(stopwords=STOPWORDS, background_color='black', width=2500, height=1800).generate(' '.join(first_topic_words))
plt.imshow(firstcloud)
plt.axis('off')

In [ ]:
# Generating the wordcloud with the values under the category dataframe
cloud = WordCloud(stopwords=STOPWORDS, background_color='black', width=2500, height=1800).generate(' '.join(second_topic_words))
plt.imshow(cloud)
plt.axis('off')

In [ ]:
# Generating the wordcloud with the values under the category dataframe
cloud = WordCloud(stopwords=STOPWORDS, background_color='black', width=2500, height=1800).generate(' '.join(third_topic_words))
plt.imshow(cloud)
plt.axis('off')

In [ ]:
# Generating the wordcloud with the values under the category dataframe
cloud = WordCloud(stopwords=STOPWORDS, background_color='black', width=2500, height=1800).generate(' '.join(fourth_topic_words))
plt.imshow(cloud)
plt.axis('off')